In [1]:
#Bag Of Words(BOW):단어들의 등장 횟수로 표현(단어가방)
#1)주어진 단어에 대해 고유의 인덱스 부여
#2)단어의 등장 횟수 벡터 생성

In [4]:
from konlpy.tag import Okt
okt=Okt()

C:\Users\Kyujin\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [5]:
import re
token="오늘은 금요일입니다. 내일은 토요일입니다. 다음주 화요일에는 특강이 있습니다."
#정규표현식 사용하여 token에 저장된 문자 중에서 (.)을 제거하세요.
token=re.sub("\.","",token)
token

'오늘은 금요일입니다 내일은 토요일입니다 다음주 화요일에는 특강이 있습니다'

In [6]:
token=okt.morphs(token)

In [7]:
#1)주어진 단어에 대해 고유의 인덱스 부여(0~11)
#2)단어의 등장 횟수 벡터 생성
#1) => {'오늘':0, ..., '있습니다':11}
#2) => [1, 2, 1, 1, 1, 2, 1, 1,1 ...]

In [8]:
word2index={}  
bow=[]  
for voc in token:
    if voc not in word2index.keys():  
             word2index[voc]=len(word2index)  
             bow.insert(len(word2index)-1,1)
    else:
            index=word2index.get(voc)
            bow[index]=bow[index]+1
print(word2index)  

{'오늘': 0, '은': 1, '금요일': 2, '입니다': 3, '내일': 4, '토요일': 5, '다음주': 6, '화요일': 7, '에는': 8, '특강': 9, '이': 10, '있습니다': 11}


In [9]:
bow

[1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
text=['you know I want your love. because I love you.']
#vec=CountVectorizer() #BOW 생성 클래스
#vec=CountVectorizer(stop_words=['I']) #불용어 사용(사용자 정의)
vec=CountVectorizer(stop_words='english') #불용어 사용(사용자 정의)
print(vec.fit_transform(text).toarray())#각 단어별 개수를 세며, 변환
print(vec.vocabulary_)


[[1 2 1]]
{'know': 0, 'want': 2, 'love': 1}


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=['you know I want your love',
       'I like you',
       'what should I do']

In [12]:
#DTM 생성
vec=CountVectorizer()
print(vec.fit_transform(corpus).toarray())
print(vec.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [13]:
#tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv=TfidfVectorizer().fit(corpus)
print(tfidfv)
print('='*50)
print(tfidfv.transform(corpus).toarray())
print('='*50)
print(tfidfv.vocabulary_)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [14]:
dtmVector=CountVectorizer()
xtrainDtm=dtmVector.fit_transform(newsdata.data)
xtrainDtm.shape

(11314, 130107)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer()
tfidfv=tfidf_transformer.fit_transform(xtrainDtm)

In [20]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(tfidfv, newsdata.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
testdata=fetch_20newsgroups(subset='test', shuffle=True)
xtestDtm=dtmVector.transform(testdata.data)
tfidf_test=tfidf_transformer.fit_transform(xtestDtm)

In [23]:
from sklearn.metrics import accuracy_score
predicted=model.predict(tfidf_test)
print(accuracy_score(testdata.target, predicted))

0.7707116303770579
